# Build parse data

In [1]:
import numpy as np
import pandas as pd
import re
from unidecode import unidecode
import pickle
import sys
sys.path.append('../../vietadminunits')
from utils import to_alphanumeric, to_key
from parse import parse_address

## Functions

In [2]:
def add_province_key(df_province, province_english, province_key):
    if not df_province[df_province.province_english==province_english].shape[0]:
        raise ValueError(f'{province_english} is not exist in province_english')
    elif df_province[(df_province.province_english==province_english) & (df_province.province_key==province_key)].shape[0]:
        raise ValueError(f'{province_key} is exist in province_key')
    
    df_new = df_province.loc[df_province.province_english==province_english].head(1)
    df_new['province_key'] = province_key
    df_province = pd.concat([df_province, df_new])
    return df_province

In [3]:
def add_district_key(df_district, province_english, district_english, district_key):
    if not df_district[df_district.province_english==province_english].shape[0]:
        raise ValueError(f'{province_english} is not exist in province_english')
    elif not df_district[(df_district.province_english==province_english) & (df_district.district_english==district_english)].shape[0]:
        raise ValueError(f'{district_english} is not exist in district_english of {province_english}')
    elif df_district[(df_district.province_english==province_english) & (df_district.district_english==district_english) & (df_district.district_key==district_key)].shape[0]:
        raise ValueError(f'{district_key} is exist in district_key of {province_english}, {district_english}')
    
    df_new = df_district.loc[(df_district.province_english==province_english) & (df_district.district_english==district_english)].head(1)
    df_new['district_key'] = district_key
    df_district = pd.concat([df_district, df_new])
    return df_district

In [4]:
def add_ward_key(df_ward, province_english, district_english, ward_english, ward_key):
    if not df_ward[df_ward.province_english==province_english].shape[0]:
        raise ValueError(f'{province_english} is not exist in province_english')
    
    elif not df_ward[(df_ward.province_english==province_english) & (df_ward.district_english==district_english)].shape[0]:
        raise ValueError(f'{district_english} is not exist in district_english of {province_english}')
    
    elif not df_ward[(df_ward.province_english==province_english) & (df_ward.district_english==district_english) & (df_ward.ward_english==ward_english)].shape[0]:
        raise ValueError(f'{ward_english} is not exist in ward_english of {province_english}, {district_english}')
    
    elif df_ward[(df_ward.province_english==province_english) & (df_ward.district_english==district_english) & (df_ward.ward_english==ward_english) & (df_ward.ward_key==ward_key)].shape[0]:
        raise ValueError(f'{ward_key} is exist in ward_key of {province_english}, {district_english}, {ward_english}')
    
    df_new = df_ward.loc[(df_ward.province_english==province_english) & (df_ward.district_english==district_english) & (df_ward.ward_english==ward_english)].head(1)
    df_new['ward_key'] = ward_key
    df_ward = pd.concat([df_ward, df_new])
    return df_ward

## Base dataset

In [5]:
df = pd.read_csv('../../data/output/vietnam_administrative_units.csv')

In [6]:
df.head()

,province,district,ward,long_province,long_district,long_ward,short_district,short_ward,province_english,district_english,ward_english,long_province_english,long_district_english,long_ward_english,short_district_english,short_ward_english,district_level,ward_level,district_level_english,ward_level_english
0,Hà Nội,Ba Đình,Phúc Xá,Thành phố Hà Nội,Quận Ba Đình,Phường Phúc Xá,Ba Đình,Phúc Xá,Ha Noi,Ba Dinh,Phuc Xa,Ha Noi City,Ba Dinh District,Phuc Xa Ward,Ba Dinh,Phuc Xa,Quận,Phường,District,Ward
1,Hà Nội,Ba Đình,Trúc Bạch,Thành phố Hà Nội,Quận Ba Đình,Phường Trúc Bạch,Ba Đình,Trúc Bạch,Ha Noi,Ba Dinh,Truc Bach,Ha Noi City,Ba Dinh District,Truc Bach Ward,Ba Dinh,Truc Bach,Quận,Phường,District,Ward
2,Hà Nội,Ba Đình,Vĩnh Phúc,Thành phố Hà Nội,Quận Ba Đình,Phường Vĩnh Phúc,Ba Đình,Vĩnh Phúc,Ha Noi,Ba Dinh,Vinh Phuc,Ha Noi City,Ba Dinh District,Vinh Phuc Ward,Ba Dinh,Vinh Phuc,Quận,Phường,District,Ward
3,Hà Nội,Ba Đình,Cống Vị,Thành phố Hà Nội,Quận Ba Đình,Phường Cống Vị,Ba Đình,Cống Vị,Ha Noi,Ba Dinh,Cong Vi,Ha Noi City,Ba Dinh District,Cong Vi Ward,Ba Dinh,Cong Vi,Quận,Phường,District,Ward
4,Hà Nội,Ba Đình,Liễu Giai,Thành phố Hà Nội,Quận Ba Đình,Phường Liễu Giai,Ba Đình,Liễu Giai,Ha Noi,Ba Dinh,Lieu Giai,Ha Noi City,Ba Dinh District,Lieu Giai Ward,Ba Dinh,Lieu Giai,Quận,Phường,District,Ward


In [7]:
df.shape

(10547, 20)

In [8]:
df['province_key'] = df['province'].apply(to_key, args=(1,))
df['district_key'] = df['short_district'].apply(to_key, args=(2,))
df['ward_key'] = df['short_ward'].apply(to_key, args=(3,))
df['district_level_english'].fillna('', inplace=True)
df['ward_level_english'].fillna('', inplace=True)

### Create keyword lists that are duplicated district_keyword

Eg: Hà Tĩnh has Huyện Kỳ Anh and Thị xã Kỳ Anh.

In [9]:
district_count = df[['province', 'long_district', 'district_key']].drop_duplicates()[['province', 'district_key']].value_counts().reset_index()
duplicated_districts = district_count[district_count['count'] > 1]['district_key'].tolist()

In [10]:
df_duplicated_districts = df[df.district_key.isin(duplicated_districts)][['long_province', 'long_district' ,'province_key','district_key']].drop_duplicates()

In [11]:
df_duplicated_districts

,long_province,long_district,province_key,district_key
5835,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,hatinh,kyanh
5867,Tỉnh Hà Tĩnh,Thị xã Kỳ Anh,hatinh,kyanh
9160,Tỉnh Tiền Giang,Thị xã Cai Lậy,tiengiang,cailay
9213,Tỉnh Tiền Giang,Huyện Cai Lậy,tiengiang,cailay
9552,Tỉnh Trà Vinh,Huyện Duyên Hải,travinh,duyenhai
9559,Tỉnh Trà Vinh,Thị xã Duyên Hải,travinh,duyenhai
9673,Tỉnh Đồng Tháp,Thành phố Cao Lãnh,dongthap,caolanh
9697,Tỉnh Đồng Tháp,Thành phố Hồng Ngự,dongthap,hongngu
9713,Tỉnh Đồng Tháp,Huyện Hồng Ngự,dongthap,hongngu
9748,Tỉnh Đồng Tháp,Huyện Cao Lãnh,dongthap,caolanh


My idea: We will use their wards to decide their district level.

To make sure this idea is valid, I will check whether they have some wards with the same ward_key

In [12]:
for district_key in duplicated_districts:
    long_districts = df[df.district_key==district_key]['long_district'].unique().tolist()
    a = []
    b = []
    long_district_a = long_districts[0]
    long_district_b = long_districts[1]
    ward_key_a = df[df.long_district==long_district_a]['ward_key'].unique().tolist()
    ward_key_b = df[df.long_district==long_district_b]['ward_key'].unique().tolist()
    common = list(set(a) & set(b))
    if common:
        print(district_key, 'has duplicated ward_key')

Nice! We also have default option if there are no ward in the address, I use Google Trend to decide default district level.

In [13]:
# Pickle
duplicated_district_province_keys = df_duplicated_districts['province_key'].unique().tolist()

# Use Google Trend and search to decide level
duplicated_district_keys = {
    'kyanh': {'default':'Town'},
    'cailay': {'default':'Town'},
    'duyenhai': {'default':'District'},
    'caolanh': {'default':'City'},
    'hongngu': {'default':'City'},
    'longmy': {'default':'District'}
}

for district_key in duplicated_districts:
    df_temp = df[df.district_key == district_key]
    level_data = {}
    levels = df_temp.district_level_english.unique().tolist()
    for district_level_english in levels:
        ward_keys = df_temp[df_temp.district_level_english==district_level_english]['ward_key'].unique().tolist()
        level_data[district_level_english] = ward_keys
        duplicated_district_keys[district_key]['levels'] = level_data

In [14]:
print('Provinces:', duplicated_district_province_keys, '\n')
print('Districts:', duplicated_district_keys)

Provinces: ['hatinh', 'tiengiang', 'travinh', 'dongthap', 'haugiang'] 

Districts: {'kyanh': {'default': 'Town', 'levels': {'District': ['kyxuan', 'kybac', 'kyphu', 'kyphong', 'kytien', 'kygiang', 'kydong', 'kykhang', 'kyvan', 'kytrung', 'kytho', 'kytay', 'kythuong', 'kyhai', 'kythu', 'kychau', 'kytan', 'lamhop', 'kyson', 'kylac'], 'Town': ['hungtri', 'kyninh', 'kyloi', 'kyha', 'kytrinh', 'kythinh', 'kyhoa', 'kyphuong', 'kylong', 'kylien', 'kynam']}}, 'cailay': {'default': 'Town', 'levels': {'Town': ['phuong1', 'phuong2', 'phuong3', 'phuong4', 'phuong5', 'myphuoctay', 'myhanhdong', 'myhanhtrung', 'tanphu', 'tanbinh', 'tanhoi', 'nhimy', 'nhiquy', 'thanhhoa', 'phuquy', 'longkhanh'], 'District': ['thanhloc', 'mythanhbac', 'phucuong', 'mythanhnam', 'phunhuan', 'binhphu', 'camson', 'phuan', 'mylong', 'longtien', 'hiepduc', 'longtrung', 'hoixuan', 'tanphong', 'tambinh', 'nguhiep']}}, 'duyenhai': {'default': 'District', 'levels': {'District': ['donxuan', 'donchau', 'longthanh', 'longkhanh', '

At this step, we collect these data to save to module data:
- `duplicated_district_province_keys`
- `duplicated_district_keys`

## Find district alias keywords

Data source: [Danh sách các đơn vị hành chính cấp huyện không còn tồn tại](https://vi.wikipedia.org/wiki/Danh_s%C3%A1ch_%C4%91%C6%A1n_v%E1%BB%8B_h%C3%A0nh_ch%C3%ADnh_c%E1%BA%A5p_huy%E1%BB%87n_c%E1%BB%A7a_Vi%E1%BB%87t_Nam#Danh_s%C3%A1ch_c%C3%A1c_%C4%91%C6%A1n_v%E1%BB%8B_h%C3%A0nh_ch%C3%ADnh_c%E1%BA%A5p_huy%E1%BB%87n_kh%C3%B4ng_c%C3%B2n_t%E1%BB%93n_t%E1%BA%A1i)

In [15]:
def check_containing_keyword(district, change):
    if change.count(district) == 2:
        return 1
    else:
        return 0

In [16]:
df_district_change = pd.read_csv('../../data/input/district_change.csv')

In [17]:
df_district_change.dropna(inplace=True)

In [18]:
df_district_change['containing_keyword_two_times'] = df_district_change.apply(lambda x: check_containing_keyword(x.old_district, x.change), axis=1)

In [19]:
df_change = df_district_change[(df_district_change.change.fillna('').str.contains('sáp nhập|đổi tên')) & ~(df_district_change.change.fillna('').str.contains('một phần'))].copy()

In [20]:
df_change

,province,old_district,year,change,containing_keyword_two_times
0,An Giang,Huệ Đức,1977.0,sáp nhập toàn bộ vào huyện Châu Thành,0
5,Bà Rịa – Vũng Tàu,Tân Thành,2018.0,"thành lập thị xã, đổi tên thành thị xã Phú Mỹ",0
7,Bắc Kạn,Chợ Rã,1984.0,đổi tên thành huyện Ba Bể,0
9,Bạc Liêu,Minh Hải,1984.0,đổi tên thành thị xã Bạc Liêu,0
14,Bình Dương,Châu Thành,1977.0,sáp nhập vào huyện Tân Uyên và thị xã Thủ Dầu Một,0
15,Bình Dương,Lái Thiêu,1977.0,sáp nhập với huyện Dĩ An thành huyện Thuận An,0
18,Bình Phước,Phước Bình,1977.0,sáp nhập với huyện Bù Đăng và huyện Bù Đốp thà...,0
23,Cà Mau,Ngọc Hiển,1984.0,đổi tên thành huyện Đầm Dơi,0
26,Cao Bằng,Phục Hòa,2020.0,sáp nhập với huyện Quảng Uyên thành huyện Quản...,0
27,Cao Bằng,Quảng Uyên,2020.0,sáp nhập với huyện Phục Hòa thành huyện Quảng Hòa,0


In [21]:
df_split = df_district_change[df_district_change.containing_keyword_two_times==1].copy()

In [22]:
df_split

,province,old_district,year,change,containing_keyword_two_times
12,Bến Tre,Mỏ Cày,2009.0,chia tách thành huyện Mỏ Cày Bắc và huyện Mỏ C...,1
17,Bình Dương,Tân Uyên,2013.0,chia tách thành thị xã Tân Uyên và huyện Bắc T...,1
21,Bình Thuận,Hàm Thuận,1982.0,chia tách thành huyện Hàm Thuận Bắc và huyện H...,1
45,Hà Nội,Từ Liêm,2013.0,chia tách thành quận Bắc Từ Liêm và Nam Từ Liêm,1
99,Quảng Nam,Trà My,2003.0,chia tách thành huyện Bắc Trà My và huyện Nam ...,1
124,Tiền Giang,Gò Công,1979.0,chia tách thành huyện Gò Công Đông và Gò Công Tây,1


In [23]:
# This is a particular case that is duplicated district_key
df_split = df_split[df_split.old_district!='Tân Uyên'].copy()

In [24]:
# Use Google Trend to decide new district
df_split.loc[df_split.old_district=='Mỏ Cày', 'district_english'] = 'Mo Cay Nam'
df_split.loc[df_split.old_district=='Hàm Thuận', 'district_english'] = 'Ham Thuan Nam'
df_split.loc[df_split.old_district=='Từ Liêm', 'district_english'] = 'Nam Tu Liem'
df_split.loc[df_split.old_district=='Trà My', 'district_english'] = 'Nam Tra My'
df_split.loc[df_split.old_district=='Gò Công', 'district_english'] = 'Go Cong Tay'

In [25]:
df_split

,province,old_district,year,change,containing_keyword_two_times,district_english
12,Bến Tre,Mỏ Cày,2009.0,chia tách thành huyện Mỏ Cày Bắc và huyện Mỏ C...,1,Mo Cay Nam
21,Bình Thuận,Hàm Thuận,1982.0,chia tách thành huyện Hàm Thuận Bắc và huyện H...,1,Ham Thuan Nam
45,Hà Nội,Từ Liêm,2013.0,chia tách thành quận Bắc Từ Liêm và Nam Từ Liêm,1,Nam Tu Liem
99,Quảng Nam,Trà My,2003.0,chia tách thành huyện Bắc Trà My và huyện Nam ...,1,Nam Tra My
124,Tiền Giang,Gò Công,1979.0,chia tách thành huyện Gò Công Đông và Gò Công Tây,1,Go Cong Tay


In [26]:
def find_new_district_english(text):
    au = parse_address(text, 2)
    return au.district_english

In [27]:
def prepare_address(province, change):
    address = re.sub(f'.* thành ', '', change) + ', ' + province
    return address

In [28]:
def create_province_english(text):
    au = parse_address(text, 1)
    return au.province_english

In [29]:
df_change['address'] = df_change.apply(lambda row: prepare_address(row.province, row.change), axis=1)

In [30]:
df_change

,province,old_district,year,change,containing_keyword_two_times,address
0,An Giang,Huệ Đức,1977.0,sáp nhập toàn bộ vào huyện Châu Thành,0,"sáp nhập toàn bộ vào huyện Châu Thành, An Giang"
5,Bà Rịa – Vũng Tàu,Tân Thành,2018.0,"thành lập thị xã, đổi tên thành thị xã Phú Mỹ",0,"thị xã Phú Mỹ, Bà Rịa – Vũng Tàu"
7,Bắc Kạn,Chợ Rã,1984.0,đổi tên thành huyện Ba Bể,0,"huyện Ba Bể, Bắc Kạn"
9,Bạc Liêu,Minh Hải,1984.0,đổi tên thành thị xã Bạc Liêu,0,"thị xã Bạc Liêu, Bạc Liêu"
14,Bình Dương,Châu Thành,1977.0,sáp nhập vào huyện Tân Uyên và thị xã Thủ Dầu Một,0,sáp nhập vào huyện Tân Uyên và thị xã Thủ Dầu ...
15,Bình Dương,Lái Thiêu,1977.0,sáp nhập với huyện Dĩ An thành huyện Thuận An,0,"huyện Thuận An, Bình Dương"
18,Bình Phước,Phước Bình,1977.0,sáp nhập với huyện Bù Đăng và huyện Bù Đốp thà...,0,"huyện Phước Long, Bình Phước"
23,Cà Mau,Ngọc Hiển,1984.0,đổi tên thành huyện Đầm Dơi,0,"huyện Đầm Dơi, Cà Mau"
26,Cao Bằng,Phục Hòa,2020.0,sáp nhập với huyện Quảng Uyên thành huyện Quản...,0,"huyện Quảng Hòa, Cao Bằng"
27,Cao Bằng,Quảng Uyên,2020.0,sáp nhập với huyện Phục Hòa thành huyện Quảng Hòa,0,"huyện Quảng Hòa, Cao Bằng"


In [31]:
df_change = df_change[df_change.old_district!='Thủ Đức (quận)'].copy()

In [32]:
df_change['district_english'] = df_change.address.apply(find_new_district_english)

sapnhaptoanbovaohuyen,
thixa,
huyen,
thixa,
sapnhapvaohuyentanuyenvathixa,
huyen,
huyen,
huyen,
huyen,
huyen,
sapnhapvaohuyen,
sapnhapvaohuyen,
thixa,
huyen,
huyen,
huyen,
thixa,
huyen,
huyen,
sapnhapvaohuyen,
huyen,
thixa,
pho,
huyen,
huyen,
sapnhapvoithixa,
pho,
huyen,
huyen,
huyen,
huyen,
huyen,
sapnhapvaohuyen,
pho,
huyen,
thixa,
thixa,
pho,
huyen,
huyen,
huyen,
thixa,
huyen,tp.
pho,tp.
pho,tp.
sapnhapvaohuyencanglongvahuyen,
huyen,
huyen,


After doing research, I found district_english `None` are fine, because they are moved to another province or revert it's changing.

In [33]:
df_change.dropna(inplace=True)

In [34]:
df_change.dropna(inplace=True)
df_all = pd.concat([df_change, df_split])
df_all['province_english'] = df_all.province.apply(create_province_english)
df_all['district_key'] = df_all.old_district.apply(to_key, args=(2,))

In [35]:
district_alias_keys = [tuple(i) for i in  df_all[['province_english', 'district_english', 'district_key']].values.tolist()]

In [36]:
district_alias_keys

[('An Giang', 'Chau Thanh', 'hueduc'),
 ('Ba Ria - Vung Tau', 'Phu My', 'tanthanh'),
 ('Bac Kan', 'Ba Be', 'chora'),
 ('Bac Lieu', 'Bac Lieu', 'minhhai'),
 ('Binh Duong', 'Thu Dau Mot', 'chauthanh'),
 ('Binh Duong', 'Thuan An', 'laithieu'),
 ('Binh Phuoc', 'Phuoc Long', 'phuocbinh'),
 ('Ca Mau', 'Dam Doi', 'ngochien'),
 ('Cao Bang', 'Quang Hoa', 'phuchoa'),
 ('Cao Bang', 'Quang Hoa', 'quanguyen'),
 ('Cao Bang', 'Ha Quang', 'thongnong'),
 ('Cao Bang', 'Trung Khanh', 'tralinh'),
 ('Dien Bien', 'Muong Lay', 'laichau'),
 ('Dien Bien', 'Muong Cha', 'muonglay'),
 ('Dong Nai', 'Vinh Cuu', 'vinhan'),
 ('Dong Thap', 'Lap Vo', 'thanhhung'),
 ('Ha Nam', 'Phu Ly', 'hanam'),
 ('Ha Noi', 'Soc Son', 'daphuc'),
 ('Ha Noi', 'Soc Son', 'kimanh'),
 ('Hai Phong', 'Cat Hai', 'catba'),
 ('Hau Giang', 'Vi Thanh', 'mythanh'),
 ('Hau Giang', 'Nga Bay', 'tanhiep'),
 ('Hoa Binh', 'Hoa Binh', 'kyson'),
 ('Khanh Hoa', 'Dien Khanh', 'khanhxuong'),
 ('Kien Giang', 'Kien Luong', 'hatien'),
 ('Lao Cai', 'Lao Cai', 'ca

In [37]:
df_split['old_district_key'] = df_split['old_district'].apply(to_key, args=(2,))

In [38]:
# Pickle
half_district_keys = {}
for old_district_key in df_split['old_district_key'].unique().tolist():
    df_temp = df[df.district_key.str.contains(old_district_key)]
    district_keys =  df_temp.district_key.unique().tolist()
    district_key_data = {}
    for district_key in district_keys:
        district_key_data[district_key] = df_temp[df_temp.district_key==district_key]['ward_key'].unique().tolist()
    half_district_keys[old_district_key] = district_key_data

In [39]:
half_district_keys

{'mocay': {'mocaynam': ['mocay',
   'dinhthuy',
   'daphuochoi',
   'tanhoi',
   'phuochiep',
   'binhkhanh',
   'anthanh',
   'andinh',
   'thanhthoib',
   'tantrung',
   'anthoi',
   'thanhthoia',
   'minhduc',
   'ngaidang',
   'camson',
   'huongmy'],
  'mocaybac': ['phumy',
   'hungkhanhtrunga',
   'thanhtan',
   'thanhngai',
   'tanphutay',
   'phuocmytrung',
   'tanthanhbinh',
   'thanhan',
   'hoaloc',
   'tanthanhtay',
   'tanbinh',
   'nhuanphutan',
   'khanhthanhtan']},
 'hamthuan': {'hamthuanbac': ['malam',
   'phulong',
   'lada',
   'dongtien',
   'thuanhoa',
   'donggiang',
   'hamphu',
   'hongliem',
   'thuanminh',
   'hongson',
   'hamtri',
   'hamduc',
   'hamliem',
   'hamchinh',
   'hamhiep',
   'hamthang',
   'dami'],
  'hamthuannam': ['thuannam',
   'mythanh',
   'hamcan',
   'muongman',
   'hamthanh',
   'hamkiem',
   'hamcuong',
   'hammy',
   'tanlap',
   'hamminh',
   'thuanquy',
   'tanthuan',
   'tanthanh']},
 'tuliem': {'namtuliem': ['caudien',
   'xuanphu

After this step, we got `half_district_keys` for module data

### Create keyword lists that are duplicated ward_keyword

In [40]:
ward_count = df[['long_province', 'long_district', 'short_ward']].value_counts().reset_index()
df_duplicated_wards = ward_count[ward_count['count'] > 1].copy()
df_duplicated_wards.sort_values(by=['long_province', 'long_district', 'short_ward'], inplace=True)

In [41]:
df_duplicated_wards = df[(df.long_province.isin(df_duplicated_wards.long_province)) & (df.long_district.isin(df_duplicated_wards.long_district)) & (df.short_ward.isin(df_duplicated_wards.short_ward))][['province', 'long_district', 'ward','ward_level_english', 'district_key', 'ward_key']]

In [42]:
df_duplicated_wards

,province,long_district,ward,ward_level_english,district_key,ward_key
176,Hà Nội,Huyện Gia Lâm,Thị trấn Yên Viên,Town,gialam,yenvien
178,Hà Nội,Huyện Gia Lâm,Xã Yên Viên,Commune,gialam,yenvien
1479,Lai Châu,Huyện Mường Tè,Thị trấn Mường Tè,Town,muongte,muongte
1484,Lai Châu,Huyện Mường Tè,Xã Mường Tè,Commune,muongte,muongte
1706,Sơn La,Huyện Mai Sơn,Thị trấn Hát Lót,Town,maison,hatlot
1716,Sơn La,Huyện Mai Sơn,Xã Hát Lót,Commune,maison,hatlot
1882,Yên Bái,Huyện Trạm Tấu,Thị trấn Trạm Tấu,Town,tramtau,tramtau
1887,Yên Bái,Huyện Trạm Tấu,Xã Trạm Tấu,Commune,tramtau,tramtau
1918,Yên Bái,Huyện Yên Bình,Thị trấn Yên Bình,Town,yenbinh,yenbinh
1937,Yên Bái,Huyện Yên Bình,Xã Yên Bình,Commune,yenbinh,yenbinh


Following Google Trend, "xã ..." is higher "thị trấn ..." significant. But "phường ..." is higher than "xã ..." and we have only one ward in the list.

In [43]:
# Pickle
duplicated_ward_keys = df_duplicated_wards.ward_key.unique().tolist()
duplicated_ward_district_keys = df_duplicated_wards.district_key.unique().tolist()

new_data = {}

for ward_key in duplicated_ward_keys:
    levels = df_duplicated_wards[df_duplicated_wards.ward_key == ward_key]['ward_level_english'].tolist()
    if 'Ward' in levels:
        ward_level = 'Ward'
    elif 'Commune' in levels:
        ward_level = 'Commune'
    else:
        ward_level = 'Town'
    new_data[ward_key] = ward_level
    
duplicated_ward_keys = new_data.copy()

In [44]:
print('Districts:' ,duplicated_ward_district_keys)
print('Wards:', duplicated_ward_keys)

Districts: ['gialam', 'muongte', 'maison', 'tramtau', 'yenbinh', 'chilang', 'dinhlap', 'damha', 'dahuoai', 'duchoa', 'tracu', 'thapmuoi', 'caolanh', 'vinhthuan', 'longphu', 'trande', 'phuoclong', 'camau', 'thoibinh']
Wards: {'yenvien': 'Commune', 'muongte': 'Commune', 'hatlot': 'Commune', 'tramtau': 'Commune', 'yenbinh': 'Commune', 'chilang': 'Commune', 'dinhlap': 'Commune', 'damha': 'Commune', 'madaguoi': 'Commune', 'hiephoa': 'Commune', 'dinhan': 'Commune', 'myan': 'Commune', 'mytho': 'Commune', 'vinhthuan': 'Commune', 'longphu': 'Commune', 'lichhoithuong': 'Commune', 'phuoclong': 'Commune', 'tanthanh': 'Ward', 'thoibinh': 'Commune'}


After this step, we got these data for module:
- `duplicated_ward_keys`
- `duplicated_ward_district_keys`

### Sort province_key to prioritize some provinces
- Some province has districts, or wards that are the same keyword with another province.
- We need to run module_testing to check whether any address is wrong > Use ChatGPT to sort keywords > Add manually if we found new wrong keyword.

For example:
- 'Huyện Quang Bình, Tỉnh Hà Giang' -> quangbinh
- 'Huyện Phù Yên, Tỉnh Sơn La' -> phuyen
- 'Huyện Văn Giang, Tỉnh Hưng Yên' -> angiang
- 'Huyện Quảng Ninh, Tỉnh Quảng Bình' -> quangninh
- Bac Lieu, Hoa Binh District -> hoabinh

In [45]:
province_keys = df.province_key.unique().tolist()

In [46]:
def find_province_key_match(text):
    for province_key in province_keys:
        if province_key in text:
            return province_key

In [47]:
match_district = df[df.district_key.str.contains('|'.join(province_keys))][['province', 'district', 'province_key','district_key']].drop_duplicates()

In [48]:
match_ward = df[(~df.ward.isna()) & (df.ward_key.str.contains('|'.join(province_keys)))][['province', 'ward', 'province_key','ward_key']].drop_duplicates()

In [49]:
match_district['match_province_key'] = match_district['district_key'].apply(find_province_key_match)

In [50]:
match_ward['match_province_key'] = match_ward['ward_key'].apply(find_province_key_match)

In [51]:
match_district = match_district[match_district.province_key != match_district.match_province_key]

In [52]:
match_district

,province,district,province_key,district_key,match_province_key
757,Hà Giang,Quang Bình,hagiang,quangbinh,quangbinh
1649,Sơn La,Phù Yên,sonla,phuyen,phuyen
3824,Hưng Yên,Văn Giang,hungyen,vangiang,angiang
5972,Quảng Bình,Quảng Ninh,quangbinh,quangninh,quangninh
9009,Long An,Thạnh Hóa,longan,thanhhoa,thanhhoa
10438,Bạc Liêu,Hòa Bình,baclieu,hoabinh,hoabinh


In [53]:
match_ward = match_ward[match_ward.province_key != match_ward.match_province_key]

In [54]:
match_ward

,province,ward,province_key,ward_key,match_province_key
2,Hà Nội,Vĩnh Phúc,hanoi,vinhphuc,vinhphuc
8,Hà Nội,Điện Biên,hanoi,dienbien,dienbien
142,Hà Nội,Xuân Giang,hanoi,xuangiang,angiang
316,Hà Nội,Yên Bài,hanoi,yenbai,yenbai
353,Hà Nội,Đồng Tháp,hanoi,dongthap,dongthap
...,...,...,...,...,...
10154,Cần Thơ,Thạnh Hòa,cantho,thanhhoa,thanhhoa
10236,Hậu Giang,Thạnh Hòa,haugiang,thanhhoa,thanhhoa
10363,Sóc Trăng,Khánh Hòa,soctrang,khanhhoa,khanhhoa
10438,Bạc Liêu,Hòa Bình,baclieu,hoabinh,hoabinh


In [55]:
# Should not use list(set()) because it will re-order
province_order = []

for row in match_district.itertuples():
    if row.province_key not in province_order:
        province_order.append(row.province_key)
    if row.match_province_key not in province_order:
        province_order.append(row.match_province_key)
        
for row in match_ward.itertuples():
    if row.province_key not in province_order:
        province_order.append(row.province_key)
    if row.match_province_key not in province_order:
        province_order.append(row.match_province_key)

In [56]:
province_key_order = {}
for index, province_key in enumerate(province_order):
    province_key_order[province_key] = index + 1
    

In [57]:
print(province_key_order)

{'hagiang': 1, 'quangbinh': 2, 'sonla': 3, 'phuyen': 4, 'hungyen': 5, 'angiang': 6, 'quangninh': 7, 'longan': 8, 'thanhhoa': 9, 'baclieu': 10, 'hoabinh': 11, 'hanoi': 12, 'vinhphuc': 13, 'dienbien': 14, 'yenbai': 15, 'dongthap': 16, 'caobang': 17, 'binhduong': 18, 'backan': 19, 'phutho': 20, 'tuyenquang': 21, 'thaibinh': 22, 'laichau': 23, 'khanhhoa': 24, 'binhthuan': 25, 'thainguyen': 26, 'langson': 27, 'bacgiang': 28, 'hanam': 29, 'binhdinh': 30, 'bacninh': 31, 'haiphong': 32, 'lamdong': 33, 'vinhlong': 34, 'tayninh': 35, 'haugiang': 36, 'namdinh': 37, 'haiduong': 38, 'ninhbinh': 39, 'nghean': 40, 'hatinh': 41, 'kiengiang': 42, 'quangtri': 43, 'thuathienhue': 44, 'danang': 45, 'quangnam': 46, 'travinh': 47, 'quangngai': 48, 'binhphuoc': 49, 'kontum': 50, 'daknong': 51, 'gialai': 52, 'daklak': 53, 'dongnai': 54, 'bariavungtau': 55, 'hochiminh': 56, 'tiengiang': 57, 'cantho': 58, 'soctrang': 59, 'camau': 60}


In [58]:
df['province_key_order'] = df['province_key'].map(province_key_order).fillna(0)

df.sort_values(by='province_key_order', inplace=True)

### Add alias province_keyword
- We should not add "hn" because it will cause many wrong matches. I will replace `\bhn\b` to `\bha noi\b` of the address, this step is in `parse.py`

In [59]:
df_province = df[[col for col in df.columns if 'province' in col]].drop_duplicates()

province_alias_keys = [
    ('Ho Chi Minh', 'hcm')
]

for key in province_alias_keys:
    province_english, province_key = key
    df_province = add_province_key(df_province, province_english, province_key)

### Add alias district_keyword
There are many district was changed it's name or be combined with other districts. For instance: Quan 9 > Thanh pho Thu Duc.

Use `find_district_alias_keywords.ipynb` to create a list of tuples.

In [60]:
df_district = df[['province_english'] + [col for col in df.columns if 'district' in col]].drop_duplicates()

for key in district_alias_keys:
    province_english, district_english, district_key = key
    df_district = add_district_key(df_district, province_english, district_english, district_key)

In [61]:
district_alias_keys

[('An Giang', 'Chau Thanh', 'hueduc'),
 ('Ba Ria - Vung Tau', 'Phu My', 'tanthanh'),
 ('Bac Kan', 'Ba Be', 'chora'),
 ('Bac Lieu', 'Bac Lieu', 'minhhai'),
 ('Binh Duong', 'Thu Dau Mot', 'chauthanh'),
 ('Binh Duong', 'Thuan An', 'laithieu'),
 ('Binh Phuoc', 'Phuoc Long', 'phuocbinh'),
 ('Ca Mau', 'Dam Doi', 'ngochien'),
 ('Cao Bang', 'Quang Hoa', 'phuchoa'),
 ('Cao Bang', 'Quang Hoa', 'quanguyen'),
 ('Cao Bang', 'Ha Quang', 'thongnong'),
 ('Cao Bang', 'Trung Khanh', 'tralinh'),
 ('Dien Bien', 'Muong Lay', 'laichau'),
 ('Dien Bien', 'Muong Cha', 'muonglay'),
 ('Dong Nai', 'Vinh Cuu', 'vinhan'),
 ('Dong Thap', 'Lap Vo', 'thanhhung'),
 ('Ha Nam', 'Phu Ly', 'hanam'),
 ('Ha Noi', 'Soc Son', 'daphuc'),
 ('Ha Noi', 'Soc Son', 'kimanh'),
 ('Hai Phong', 'Cat Hai', 'catba'),
 ('Hau Giang', 'Vi Thanh', 'mythanh'),
 ('Hau Giang', 'Nga Bay', 'tanhiep'),
 ('Hoa Binh', 'Hoa Binh', 'kyson'),
 ('Khanh Hoa', 'Dien Khanh', 'khanhxuong'),
 ('Kien Giang', 'Kien Luong', 'hatien'),
 ('Lao Cai', 'Lao Cai', 'ca

Some districts contain "quan" with number in their keywords. We need to make a copy and replace "quan" to "district".

In [62]:
hcm_districts = df_district[df_district['district_key'].str.contains(r'quan\d{1,2}')].copy()
hcm_districts['district_key'] = hcm_districts['district_key'].str.replace('quan', 'district')
df_district = pd.concat([df_district, hcm_districts])
hcm_districts['district_key'] = hcm_districts['district_key'].str.replace('district', 'q.')
df_district = pd.concat([df_district, hcm_districts])

### Create alias ward_key
Some wards contain "phuong" with number in their keywords. We need to make a copy and replace "phuong" to "ward".


In [63]:
df_ward = df[['province_english', 'district_english'] + [col for col in df.columns if 'ward' in col]].drop_duplicates()

In [64]:
number_wards = df_ward[df_ward['ward_key'].fillna('').str.contains(r'phuong\d{1,2}')].copy()
number_wards['ward_key'] = number_wards['ward_key'].str.replace('phuong', 'ward')
df_ward = pd.concat([df_ward, number_wards])
number_wards['ward_key'] = number_wards['ward_key'].str.replace('ward', 'p.')
df_ward = pd.concat([df_ward, number_wards])

### Create province_keys lists
We will prioritize provinces that are not in district_key or ward_key when searching a province_key in address.

In [65]:
province_keys = df_province['province_key'].tolist()

# Pickle
province_keys_1 = []
province_keys_2 = []
province_keys_3 = []

district_keys = str(df.district_key.unique().tolist())
ward_keys = str(df.ward_key.unique().tolist())
for province_key in province_keys:
    if (province_key not in district_keys) and (province_key not in ward_keys):
        province_keys_1.append(province_key)
    elif province_key not in ward_keys:
        province_keys_2.append(province_key)
    else:
        province_keys_3.append(province_key)

After this step we got these data for module:
- `province_keys_1`
- `province_keys_2`
- `province_keys_3`

### Create mapping dictionaries


In [66]:
# Pickle
province_map = {}

for province_key in df_province.province_key.unique():
    province = df_province[df_province['province_key'] == province_key]
    province_record = province.to_dict(orient='records')[0]
    province_map[province_key] = province_record

In [67]:
print(province_map)

{'laocai': {'province': 'Lào Cai', 'long_province': 'Tỉnh Lào Cai', 'province_english': 'Lao Cai', 'long_province_english': 'Lao Cai Province', 'province_key': 'laocai', 'province_key_order': 0.0}, 'bentre': {'province': 'Bến Tre', 'long_province': 'Tỉnh Bến Tre', 'province_english': 'Ben Tre', 'long_province_english': 'Ben Tre Province', 'province_key': 'bentre', 'province_key_order': 0.0}, 'ninhthuan': {'province': 'Ninh Thuận', 'long_province': 'Tỉnh Ninh Thuận', 'province_english': 'Ninh Thuan', 'long_province_english': 'Ninh Thuan Province', 'province_key': 'ninhthuan', 'province_key_order': 0.0}, 'hagiang': {'province': 'Hà Giang', 'long_province': 'Tỉnh Hà Giang', 'province_english': 'Ha Giang', 'long_province_english': 'Ha Giang Province', 'province_key': 'hagiang', 'province_key_order': 1.0}, 'quangbinh': {'province': 'Quảng Bình', 'long_province': 'Tỉnh Quảng Bình', 'province_english': 'Quang Binh', 'long_province_english': 'Quang Binh Province', 'province_key': 'quangbinh', 

In [68]:
# Pickle
district_map = {}

for province_english in df_province.province_english.unique():
    district_keys = {}
    for district_key in df_district[df_district.province_english == province_english]['district_key'].unique():
        district_levels = {}
        for district_level_english in df_district[(df_district.province_english == province_english) & (df_district.district_key == district_key)]['district_level_english'].unique():
            district = df_district[(df_district.province_english == province_english) & (df_district.district_key == district_key) & (df_district.district_level_english == district_level_english)]
            district_record = district[[col for col in district.columns if 'district' in col]].to_dict('records')[0]
            district_levels[district_level_english] = district_record
        district_keys[district_key] = district_levels
        
    district_keys = dict(sorted(district_keys.items(), key=lambda item: len(item[0]), reverse=True))
    
    district_map[province_english] = district_keys

In [69]:
print(district_map['Ho Chi Minh'])

{'district10': {'District': {'district': 'Quận 10', 'long_district': 'Quận 10', 'short_district': 'Quận 10', 'district_english': 'District 10', 'long_district_english': 'District 10', 'short_district_english': 'District 10', 'district_level': 'Quận', 'district_level_english': 'District', 'district_key': 'district10'}}, 'district11': {'District': {'district': 'Quận 11', 'long_district': 'Quận 11', 'short_district': 'Quận 11', 'district_english': 'District 11', 'long_district_english': 'District 11', 'short_district_english': 'District 11', 'district_level': 'Quận', 'district_level_english': 'District', 'district_key': 'district11'}}, 'district12': {'District': {'district': 'Quận 12', 'long_district': 'Quận 12', 'short_district': 'Quận 12', 'district_english': 'District 12', 'long_district_english': 'District 12', 'short_district_english': 'District 12', 'district_level': 'Quận', 'district_level_english': 'District', 'district_key': 'district12'}}, 'binhthanh': {'District': {'district': 

In [70]:
# Pickle
ward_map = {}

for province_english in df_province.province_english.unique():
    districts = {}
    for district_english in df_district[df_district.province_english==province_english].district_english.unique():
        wards = {}
        for ward_key in df_ward[(df_ward.province_english==province_english) & (df_ward.district_english==district_english)].ward_key.dropna().unique():
            ward_levels = {}
            for ward_level_english in df_ward[(df_ward.province_english==province_english) & (df_ward.district_english==district_english) & (df_ward.ward_key==ward_key)].ward_level_english.unique():
                ward = df_ward[(df_ward.province_english==province_english) & (df_ward.district_english==district_english) & (df_ward.ward_key==ward_key) & (df_ward.ward_level_english==ward_level_english)]
                ward_record = ward[[col for col in ward.columns if 'ward' in col]].to_dict('records')[0]
                ward_levels[ward_level_english] = ward_record
            wards[ward_key] = ward_levels
        wards = dict(sorted(wards.items(), key=lambda item: len(str(item[0])), reverse=True))
        districts[district_english] = wards
    ward_map[province_english] = districts

In [71]:
print(ward_map['Ho Chi Minh']['Tan Binh'])

{'phuong15': {'Ward': {'ward': 'Phường 15', 'long_ward': 'Phường 15', 'short_ward': 'Phường 15', 'ward_english': 'Ward 15', 'long_ward_english': 'Ward 15', 'short_ward_english': 'Ward 15', 'ward_level': 'Phường', 'ward_level_english': 'Ward', 'ward_key': 'phuong15'}}, 'phuong14': {'Ward': {'ward': 'Phường 14', 'long_ward': 'Phường 14', 'short_ward': 'Phường 14', 'ward_english': 'Ward 14', 'long_ward_english': 'Ward 14', 'short_ward_english': 'Ward 14', 'ward_level': 'Phường', 'ward_level_english': 'Ward', 'ward_key': 'phuong14'}}, 'phuong10': {'Ward': {'ward': 'Phường 10', 'long_ward': 'Phường 10', 'short_ward': 'Phường 10', 'ward_english': 'Ward 10', 'long_ward_english': 'Ward 10', 'short_ward_english': 'Ward 10', 'ward_level': 'Phường', 'ward_level_english': 'Ward', 'ward_key': 'phuong10'}}, 'phuong11': {'Ward': {'ward': 'Phường 11', 'long_ward': 'Phường 11', 'short_ward': 'Phường 11', 'ward_english': 'Ward 11', 'long_ward_english': 'Ward 11', 'short_ward_english': 'Ward 11', 'ward_l

After this step, we got these data for module:
- `province_map`
- `district_map`
- `ward_map`

### Create double check dictionaries
Some address will cause wrong mapping because they have ward_key is as same as other province's province_key.
- Use **sub_2_create_double_check_keywords.ipynb** to get the data.
- Note that after saving this data to module, we can not create the data again.

In [72]:
double_check_provinces = {
    'angiang': ['thainguyen', 'ninhbinh', 'thanhhoa'],
    'hoabinh': ['thainguyen',
    'langson',
    'haiphong',
    'thaibinh',
    'dongnai',
    'vinhlong',
    'dongthap'],
    'binhthuan': ['thainguyen', 'quangngai', 'binhdinh'],
    'thanhhoa': ['langson', 'binhphuoc', 'travinh', 'kiengiang', 'haugiang'],
    'thaibinh': ['langson', 'tayninh'],
    'hagiang': ['thaibinh', 'thanhhoa'],
    'quangbinh': ['thaibinh', 'thanhhoa'],
    'sonla': ['ninhbinh', 'quangngai', 'khanhhoa'],
    'khanhhoa': ['ninhbinh', 'kiengiang'],
    'longan': ['thanhhoa', 'dongnai', 'vinhlong'],
    'quangninh': ['thanhhoa'],
    'vinhlong': ['quangtri'],
    'haiduong': ['quangtri'],
    'haiphong': ['quangtri'],
    'binhduong': ['quangnam', 'quangngai', 'binhdinh'],
    'binhdinh': ['quangnam'],
    'hungyen': ['kiengiang']
}

# double_check_provinces = {} # testing

Some address will cause wrong mapping because they have ward_key is as same as other district's district_key.

In [73]:
double_check_districts = {'unghoa': ['caugiay', 'chuongmy'],
 'thanhtri': ['hoangmai'],
 'thanhxuan': ['socson'],
 'thanhoai': ['thanhtri'],
 'thongnong': ['haquang'],
 'tralinh': ['trungkhanh'],
 'quanguyen': ['quanghoa'],
 'chora': ['babe'],
 'yenson': ['sonduong'],
 'camduong': ['laocai'],
 'muongla': ['phuyen', 'songma', 'sopcop'],
 'tranyen': ['lucyen', 'yenbinh'],
 'kyson': ['hoabinh', 'tanky'],
 'chilang': ['langson', 'trangdinh'],
 'honggai': ['halong'],
 'hoanhbo': ['halong'],
 'uongbi': ['mongcai'],
 'halong': ['vandon'],
 'haiduong': ['binhgiang'],
 'anduong': ['lechan', 'duongkinh', 'thuynguyen', 'vinhbao'],
 'catba': ['cathai'],
 'phucu': ['hungyen'],
 'myloc': ['namdinh'],
 'dongson': ['thanhhoa', 'bimson'],
 'hatrung': ['bathuoc'],
 'thanhhoa': ['nhuxuan', 'benluc'],
 'vinh': ['anhson', 'yenthanh', 'hoangmai'],
 'yenthanh': ['thanhchuong'],
 'anhson': ['thanhchuong', 'namdan'],
 'thachha': ['hatinh'],
 'huongthuy': ['hue'],
 'giang': ['thangbinh', 'bactramy', 'nuithanh'],
 'tramy': ['bactramy'],
 'sontinh': ['sontay'],
 'ducpho': ['moduc'],
 'dakto': ['konray', 'tumorong'],
 'dakha': ['tumorong'],
 'dakpo': ['kongchro'],
 'iapa': ['chuse'],
 'krongbuk': ['krongpac'],
 'phuocbinh': ['phuoclong'],
 'hoathanh': ['chauthanh'],
 'laithieu': ['thuanan'],
 'thongnhat': ['bienhoa'],
 'vinhan': ['vinhcuu'],
 'binhchanh': ['thuduc'],
 'tanphu': ['thuduc', 'quan7', 'cuchi'],
 'tanthanh': ['mochoa', 'thuthua'],
 'tanan': ['canduoc'],
 'tanhung': ['chauthanh'],
 'tanphuoc': ['gocongdong'],
 'mocay': ['mocaynam'],
 'thanhphu': ['giongtrom', 'binhdai'],
 'cainhum': ['mangthit'],
 'longho': ['mangthit'],
 'chauphu': ['chaudoc'],
 'anphu': ['phutan', 'tinhbien', 'chauthanh', 'thoaison'],
 'anminh': ['chauthanh', 'uminhthuong'],
 'vithanh': ['vithuy']}

# double_check_districts = {} # tesing

## Support find province from unique districts

Điều kiện 1: district_key not in ward_keys.
Điều kiện 2: district_count = 1, sau khi đã drop_duplicated [province_key, district_key]
Điều kiện 3: district_key not in province_keys của tỉnh khác.

Nếu district_key = province_key của chính nó:
>> Lấy danh sách long_district_alphanumerices (bao gồm Việt và English), nếu tồn tại trong address thì sau khi tìm được province_key sẽ không bị xóa province_key trong địa chỉ.


Nếu district_key != province_key của chính nó, tức là unique tuyệt đối:
>> Lấy danh sách unique_district_keys, nếu không tìm được province_key thì tìm unique_district_key để suy ngược ra province_key

In [74]:
df = pd.read_csv('../../data/output/vietnam_administrative_units.csv')
df['province_key'] = df['province'].apply(lambda x: to_key(x, 1))
df['district_key'] = df['short_district'].apply(lambda x: to_key(x, 2))
df['ward_key'] = df['short_ward'].apply(lambda x: to_key(x, 3))
df['ward_alphanumeric'] = df['long_ward'].apply(to_alphanumeric)


province_keys = df['province_key'].unique().tolist()
district_keys = df['district_key'].unique().tolist()
ward_keys = df['ward_key'].unique().tolist()
ward_alphanumerics = df['ward_alphanumeric'].unique().tolist()
str_ward_alphanumerics = str(ward_alphanumerics)

In [75]:
# Add alias key. Eg: Quan 2, Quan 9 are Thu Duc
for key in district_alias_keys:
    province_english, district_english, district_key = key
    df = add_district_key(df, province_english, district_english, district_key)

In [76]:
df_district_filter = df[~df.district_key.apply(lambda x: x in str_ward_alphanumerics)]

In [77]:
df_district_filter_count = df_district_filter[['province_key', 'district_key']].drop_duplicates()['district_key'].value_counts().reset_index()
one_district_keys = df_district_filter_count[df_district_filter_count['count']==1]['district_key'].tolist()
df_district_filter = df_district_filter[df_district_filter.district_key.isin(one_district_keys)][['province_key', 'district_key', 'long_district', 'long_district_english']].drop_duplicates()

def check_valid_district_key(district_key, province_key):
    if district_key in province_keys and district_key != province_key:
        return False
    else:
        return True
    
df_district_filter['is_valid_district_key'] = df_district_filter.apply(lambda x: check_valid_district_key(x['district_key'], x['province_key']), axis=1)
df_district_filter = df_district_filter[df_district_filter.is_valid_district_key]
df_district_filter['same_key'] = np.where(df_district_filter.province_key == df_district_filter.district_key, True, False)

long_district_alphanumerics = []
for row in df_district_filter[df_district_filter.same_key].itertuples():
    long_district_alphanumerics.append(to_alphanumeric(row.long_district))
    long_district_alphanumerics.append(to_alphanumeric(row.long_district).replace('thanhpho', 'tp.'))
    long_district_alphanumerics.append(to_alphanumeric(row.long_district_english))
    
    
unique_district_keys = {}
for row in df_district_filter[~df_district_filter.same_key][['province_key', 'district_key']].drop_duplicates().itertuples():
    unique_district_keys[row.district_key] = row.province_key
    district_key = row.district_key
    if re.search(r'^quan\d{1,2}', district_key):
        district_key = re.sub('^quan', 'q.', district_key)
        unique_district_keys[district_key] = row.province_key

In [78]:
len(long_district_alphanumerics)

36

In [79]:
long_district_alphanumerics

['thanhphocaobang',
 'tp.caobang',
 'caobangcity',
 'thanhphobackan',
 'tp.backan',
 'backancity',
 'thanhphotuyenquang',
 'tp.tuyenquang',
 'tuyenquangcity',
 'thanhphobacgiang',
 'tp.bacgiang',
 'bacgiangcity',
 'thanhphobacninh',
 'tp.bacninh',
 'bacninhcity',
 'thanhphonamdinh',
 'tp.namdinh',
 'namdinhcity',
 'thanhphohatinh',
 'tp.hatinh',
 'hatinhcity',
 'thanhphokontum',
 'tp.kontum',
 'kontumcity',
 'thanhphobentre',
 'tp.bentre',
 'bentrecity',
 'thanhphosoctrang',
 'tp.soctrang',
 'soctrangcity',
 'thanhphobaclieu',
 'tp.baclieu',
 'baclieucity',
 'thanhphocamau',
 'tp.camau',
 'camaucity']

In [80]:
len(unique_district_keys)

212

In [81]:
unique_district_keys

{'hoankiem': 'hanoi',
 'caugiay': 'hanoi',
 'namtuliem': 'hanoi',
 'bactuliem': 'hanoi',
 'thachthat': 'hanoi',
 'chuongmy': 'hanoi',
 'bacme': 'hagiang',
 'hoangsuphi': 'hagiang',
 'bacquang': 'hagiang',
 'haquang': 'caobang',
 'thachan': 'caobang',
 'pacnam': 'backan',
 'babe': 'backan',
 'bachthong': 'backan',
 'chodon': 'backan',
 'nari': 'backan',
 'lambinh': 'tuyenquang',
 'chiemhoa': 'tuyenquang',
 'hamyen': 'tuyenquang',
 'dienbienphu': 'dienbien',
 'muonglay': 'dienbien',
 'nampo': 'dienbien',
 'quynhnhai': 'sonla',
 'lucyen': 'yenbai',
 'vanchan': 'yenbai',
 'yenthuy': 'hoabinh',
 'lacthuy': 'hoabinh',
 'songcong': 'thainguyen',
 'donghy': 'thainguyen',
 'vonhai': 'thainguyen',
 'phoyen': 'thainguyen',
 'trangdinh': 'langson',
 'mongcai': 'quangninh',
 'campha': 'quangninh',
 'lucnam': 'bacgiang',
 'lucngan': 'bacgiang',
 'yendung': 'bacgiang',
 'vietyen': 'bacgiang',
 'viettri': 'phutho',
 'quevo': 'bacninh',
 'chilinh': 'haiduong',
 'kinhmon': 'haiduong',
 'hongbang': 'haip

## Support find province from unique "long" districts

In [82]:
df['province_alphanumeric'] = df['long_province'].apply(to_alphanumeric)
df['district_alphanumeric'] = df['long_district'].apply(to_alphanumeric)
df['district_alphanumeric_english'] = df['long_district_english'].apply(to_alphanumeric)
df['ward_alphanumeric'] = df['long_ward'].apply(to_alphanumeric)

In [83]:
province_alphanumerics = df['province_alphanumeric'].unique().tolist()
district_alphanumerics = df['district_alphanumeric'].unique().tolist()
ward_alphanumerics = df['ward_alphanumeric'].unique().tolist()

In [84]:
df_district_filter = df[~df.district_alphanumeric.isin(ward_alphanumerics)]
df_district_filter_count = df_district_filter[['province_alphanumeric', 'district_alphanumeric']].drop_duplicates()['district_alphanumeric'].value_counts().reset_index()
one_district_alphanumerics = df_district_filter_count[df_district_filter_count['count']==1]['district_alphanumeric'].tolist()
df_district_filter = df_district_filter[df_district_filter.district_alphanumeric.isin(one_district_alphanumerics)][['province_key','province_alphanumeric', 'district_alphanumeric', 'long_district', 'long_district_english', 'district_key', 'district_alphanumeric_english']].drop_duplicates()

def check_valid_district_alphanumeric(district_alphanumeric, province_alphanumeric):
    if district_alphanumeric in province_alphanumerics and district_alphanumeric != province_alphanumeric:
        return False
    else:
        return True
    
df_district_filter['is_valid_district_alphanumeric'] = df_district_filter.apply(lambda x: check_valid_district_alphanumeric(x['district_alphanumeric'], x['province_alphanumeric']), axis=1)
df_district_filter = df_district_filter[df_district_filter.is_valid_district_alphanumeric]
df_district_filter['same_alphanumeric'] = np.where(df_district_filter.province_alphanumeric == df_district_filter.district_alphanumeric, True, False)

for row in df_district_filter[df_district_filter.same_alphanumeric].itertuples():
    long_district_alphanumerics.append(to_alphanumeric(row.long_district))
    long_district_alphanumerics.append(to_alphanumeric(row.long_district).replace('thanhpho', 'tp.'))
    long_district_alphanumerics.append(to_alphanumeric(row.long_district_english))
    
    
for row in df_district_filter[~df_district_filter.same_alphanumeric][['province_key', 'district_alphanumeric', 'district_key', 'district_alphanumeric_english']].drop_duplicates().itertuples():
    if not row.district_key in unique_district_keys:
        unique_district_keys[row.district_alphanumeric] = row.province_key
        unique_district_keys[row.district_alphanumeric_english] = row.province_key
        district_alphanumeric = row.district_alphanumeric
        district_alphanumeric = re.sub('^quan', 'q.', district_alphanumeric)
        district_alphanumeric = re.sub('^huyen', 'h.', district_alphanumeric)
        district_alphanumeric = re.sub('^thanhpho', 'tp.', district_alphanumeric)
        district_alphanumeric = re.sub('^thixa', 'tx.', district_alphanumeric)
        unique_district_keys[district_alphanumeric] = row.province_key

In [85]:
len(unique_district_keys)

1683

In [86]:
len(long_district_alphanumerics)

36

In [87]:
print(unique_district_keys)

{'hoankiem': 'hanoi', 'caugiay': 'hanoi', 'namtuliem': 'hanoi', 'bactuliem': 'hanoi', 'thachthat': 'hanoi', 'chuongmy': 'hanoi', 'bacme': 'hagiang', 'hoangsuphi': 'hagiang', 'bacquang': 'hagiang', 'haquang': 'caobang', 'thachan': 'caobang', 'pacnam': 'backan', 'babe': 'backan', 'bachthong': 'backan', 'chodon': 'backan', 'nari': 'backan', 'lambinh': 'tuyenquang', 'chiemhoa': 'tuyenquang', 'hamyen': 'tuyenquang', 'dienbienphu': 'dienbien', 'muonglay': 'dienbien', 'nampo': 'dienbien', 'quynhnhai': 'sonla', 'lucyen': 'yenbai', 'vanchan': 'yenbai', 'yenthuy': 'hoabinh', 'lacthuy': 'hoabinh', 'songcong': 'thainguyen', 'donghy': 'thainguyen', 'vonhai': 'thainguyen', 'phoyen': 'thainguyen', 'trangdinh': 'langson', 'mongcai': 'quangninh', 'campha': 'quangninh', 'lucnam': 'bacgiang', 'lucngan': 'bacgiang', 'yendung': 'bacgiang', 'vietyen': 'bacgiang', 'viettri': 'phutho', 'quevo': 'bacninh', 'chilinh': 'haiduong', 'kinhmon': 'haiduong', 'hongbang': 'haiphong', 'duongkinh': 'haiphong', 'thuynguye

## Support find province from "not unique" district_key

Now we can parse "Phuong 15 Quan Tan Binh" but "Phuong 15 Tan Binh" is not, because "tanbinh" is a ward_key of another province.

To reach this feature, we will check if any ward_key of the district is in the address, and the ward_key must be placed before the district_key.

- Condition 1: ward_key_district_key is unique.
- Condition 2: district_key not in `unique_district_keys`

In [88]:
# Data I want
{
    'tanbinh': {
        'hochiminh': ['phuong1', 'phuong2', 'p.1', 'p.2', 'district1', 'district2'],
        'otherprovincekey': [...]
    }
}

{'tanbinh': {'hochiminh': ['phuong1',
   'phuong2',
   'p.1',
   'p.2',
   'district1',
   'district2'],
  'otherprovincekey': [Ellipsis]}}

In [89]:
df['ward_key_district_key'] = df['ward_key'] + '_' + df['district_key']

In [90]:
ward_key_district_key_count = df['ward_key_district_key'].value_counts().reset_index()

In [91]:
unique_ward_key_district_keys = ward_key_district_key_count[ward_key_district_key_count['count']==1]['ward_key_district_key'].tolist()

In [92]:
df_unique_ward_key_district_keys = df[(df['ward_key_district_key'].isin(unique_ward_key_district_keys)) & (~df['district_key'].isin(unique_district_keys))]

In [93]:
not_unique_district_keys = {}
for district_key in df_unique_ward_key_district_keys['district_key'].unique().tolist():
    data = {}
    tmp_province_keys = df_unique_ward_key_district_keys[df_unique_ward_key_district_keys['district_key']==district_key]['province_key'].unique().tolist()
    for province_key in tmp_province_keys:
        tmp_ward_keys = df_unique_ward_key_district_keys[(df_unique_ward_key_district_keys['district_key']==district_key) & (df_unique_ward_key_district_keys['province_key']==province_key)]['ward_key'].unique().tolist()
        tmp_number_wards = [i for i in tmp_ward_keys if re.search('^phuong\d{1,2}', i)]
        for num_ward_key in tmp_number_wards:
            tmp_ward_keys.append(num_ward_key.replace('phuong','district'))
            tmp_ward_keys.append(num_ward_key.replace('phuong','p.'))
        data[province_key] = tmp_ward_keys
    not_unique_district_keys[district_key] = data

In [94]:
not_unique_district_keys['tanbinh']

{'hochiminh': ['phuong2',
  'phuong4',
  'phuong12',
  'phuong13',
  'phuong1',
  'phuong3',
  'phuong11',
  'phuong7',
  'phuong5',
  'phuong10',
  'phuong6',
  'phuong8',
  'phuong9',
  'phuong14',
  'phuong15',
  'district2',
  'p.2',
  'district4',
  'p.4',
  'district12',
  'p.12',
  'district13',
  'p.13',
  'district1',
  'p.1',
  'district3',
  'p.3',
  'district11',
  'p.11',
  'district7',
  'p.7',
  'district5',
  'p.5',
  'district10',
  'p.10',
  'district6',
  'p.6',
  'district8',
  'p.8',
  'district9',
  'p.9',
  'district14',
  'p.14',
  'district15',
  'p.15']}

In [95]:
not_unique_district_keys['chauthanh']

{'tayninh': ['haoduoc',
  'phuocvinh',
  'dongkhoi',
  'thaibinh',
  'anco',
  'biengioi',
  'hoathanh',
  'tribinh',
  'hoahoi',
  'anbinh',
  'thanhdien',
  'thanhlong',
  'ninhdien',
  'longvinh'],
 'longan': ['tamvu',
  'binhquoi',
  'hoaphu',
  'phungaitri',
  'vinhcong',
  'thuanmy',
  'phuoctanhung',
  'thanhphulong',
  'duongxuanhoi',
  'anluclong',
  'longtri',
  'thanhvinhdong'],
 'tiengiang': ['tanhiep',
  'tanhoidong',
  'tanhuong',
  'tanlydong',
  'tanlytay',
  'thancuunghia',
  'tamhiep',
  'diemhy',
  'nhibinh',
  'duongdiem',
  'donghoa',
  'longdinh',
  'huudao',
  'longan',
  'longhung',
  'binhtrung',
  'thanhphu',
  'banlong',
  'vinhkim',
  'binhduc',
  'songthuan',
  'kimson',
  'phuphong'],
 'bentre': ['tanthach',
  'quoison',
  'giaolong',
  'phutuc',
  'phuduc',
  'phuanhoa',
  'anphuoc',
  'tamphuoc',
  'thanhtrieu',
  'tuongda',
  'quoithanh',
  'phuocthanh',
  'tienlong',
  'huudinh',
  'tienthuy',
  'sonhoa'],
 'travinh': ['daloc',
  'mychanh',
  'thanhmy'

## Support 

In [96]:
with open('../../vietadminunits/data/parse.pkl', 'wb') as f:
    pickle.dump((duplicated_district_keys, duplicated_district_province_keys, duplicated_ward_keys, duplicated_ward_district_keys, province_keys_1, province_keys_2, province_keys_3, province_map, district_map, ward_map, double_check_provinces, double_check_districts, half_district_keys, long_district_alphanumerics, unique_district_keys, not_unique_district_keys), f)